In [1]:
import os
import json
import torch
import utils
import numpy as np
from tqdm import tqdm
from collections import Counter

Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
                 It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe


In [6]:
#Load the cui data
# cuidata, _ = utils.split_cui_present_absent(True)
DATA_PATH = "P:\ORD_Singh_201911038D\Justin\Stroke_Notes_13OCT21"
with open("filtered_sids_to_cuis.json",'r') as infile:
    cuidata = json.load(infile)

In [7]:
#print(list(cuidata)[0]) # print the first document ID
print(cuidata[list(cuidata)[0]][:10]) # show the first 10 cuis of this specific document

['C0008031', 'C1507320', 'C0015031', 'C4718442', 'C2707412', 'C0281822', 'C0281822', 'C0398266', 'C3244243', 'C0917798']


In [8]:
# We will get training pairs. One is the context term the next is the target term.
trainingpairs = []
vocab = dict()
frequency = Counter()
idx = 0

# This could absolutely be accomplished in an alternative / faster way but it's sufficiently fast for now
for doc in tqdm(cuidata):
    cuis = cuidata[doc]
    cuiids = []
    for x in cuis:
        if x not in vocab.keys():
            vocab.update({x:idx})
            idx+=1
        cuiids += [vocab[x]]
    for i in range(len(cuiids)-1):
        pair = cuiids[i:i+2]
        if len(pair) < 2:
            continue
        if pair[0] != pair[1]:
            trainingpairs += [pair] #no self references
            frequency.update(pair)
print(trainingpairs[:3])
print(len(trainingpairs))
print(len(vocab))
assert np.all(np.asarray([x for x in vocab.values()]) == np.arange(len(vocab)))
# We'll use an embedding dimension of 50 to start
EMBEDDING_DIM = 100

100%|██████████████████████████████████████████████████████████████████████████| 29865/29865 [00:25<00:00, 1168.33it/s]

[[0, 1], [1, 2], [2, 3]]
3038627
111303


In [9]:
frequency = np.asarray([values for key,values in frequency.items()])**0.75
frequency /= np.linalg.norm(frequency, ord=1)
frequency = torch.from_numpy(frequency)
samplingids = torch.arange(0, len(frequency))

In [10]:
class CUIEmbeddingModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CUIEmbeddingModel, self).__init__()
        self.dim = embedding_dim
        self.in_embeddings = torch.nn.Embedding(vocab_size, embedding_dim, sparse=True)
        self.out_embeddings = torch.nn.Embedding(vocab_size, embedding_dim, sparse=True)
        
    def init(self):
        self.in_embeddings.weight.data.uniform_(-0.5/self.dim, 0.5/self.dim) # scaled by dimensionality to control initial norm.
        self.out_embeddings.weight.data.uniform(-0, 0) # all 0s
        
    def forward(self, inputs, targets, negatives):
        inembed = self.in_embeddings(inputs)
        outembed = self.out_embeddings(targets)
        pos_score = torch.sum(torch.mul(inembed, outembed), dim=1)
        pos_score = torch.nn.functional.logsigmoid(pos_score)
        negembed = self.out_embeddings(negatives)
        neg_score = torch.bmm(negembed, inembed.unsqueeze(2)).squeeze()
        neg_score = torch.nn.functional.logsigmoid(-1*neg_score)
        # skip gram negative sampling
        return -1 * (torch.sum(pos_score)+torch.sum(neg_score))
    

In [11]:
losses = []
model = CUIEmbeddingModel(len(vocab), EMBEDDING_DIM)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
NEGSAMPLES = 2

In [12]:
#Test batching (also gives you an idea of theoretical max throughput in a way)
BATCH_SIZE=2000
for batchidx in tqdm(np.arange(0, len(trainingpairs), BATCH_SIZE), total = len(trainingpairs)//BATCH_SIZE):
    data = trainingpairs[batchidx:batchidx+BATCH_SIZE]
    context_idxs = torch.tensor(data)

1520it [00:01, 901.67it/s]                                                                                             


In [13]:
def nearest_embedding_search(cui):
    #C0948008 = ischemic stroke
    for i in zip(*torch.topk(torch.nn.functional.cosine_similarity(model.in_embeddings.weight.data[vocab[cui]].view(1,-1), 
                                                                   model.in_embeddings.weight.data), 10, largest=True)):
        try:
            print(f'{i[0]:0.3f}\t{cuitranslate[id2vocab[int(i[1])]]}')
        except:
            print("key error", id2vocab[int(i[1])])

In [14]:
id2vocab = {value:key for key,value in vocab.items()}
with open(os.path.join(DATA_PATH, "cuitranslate.json"),'r') as infile:
    cuitranslate = json.load(infile)

In [15]:
print(list(id2vocab.items())[0:10])
print(list(cuitranslate.items())[0:10])
print(cuitranslate["C0000005"])

[(0, 'C0008031'), (1, 'C1507320'), (2, 'C0015031'), (3, 'C4718442'), (4, 'C2707412'), (5, 'C0281822'), (6, 'C0398266'), (7, 'C3244243'), (8, 'C0917798'), (9, 'C0013227')]
[('C0000005', '(131)I-Macroaggregated Albumin'), ('C0000039', '1,2-dipalmitoylphosphatidylcholine'), ('C0000052', '1,4-alpha-Glucan Branching Enzyme'), ('C0000074', '1-Alkyl-2-Acylphosphatidates'), ('C0000084', '1-Carboxyglutamic Acid'), ('C0000096', '1-Methyl-3-isobutylxanthine'), ('C0000097', '1-Methyl-4-phenyl-1,2,3,6-tetrahydropyridine'), ('C0000098', '1-Methyl-4-phenylpyridinium'), ('C0000102', '1-Naphthylamine'), ('C0000103', '1-Naphthylisothiocyanate')]
(131)I-Macroaggregated Albumin


In [16]:
trainingpairs = torch.LongTensor(trainingpairs)
for epoch in range(50):
    total_loss = 0
    shuffler = torch.randperm(trainingpairs.shape[0])
    trainingpairs = trainingpairs[shuffler].view(trainingpairs.size())
    with tqdm(np.arange(0, len(trainingpairs), BATCH_SIZE), desc=f'Epoch {epoch+1}', total = len(trainingpairs)//BATCH_SIZE) as progress: #goes one example at a time
        for batchidx in progress:
            data = trainingpairs[batchidx:batchidx+BATCH_SIZE]
            inputs = torch.cat((data[:, 0], data[:,1])) #we'll go bidirectional; usually not done I suppose
            targets = torch.cat((data[:, 1], data[:,0]))#doubles the batch size
            negatives = samplingids[frequency.multinomial(num_samples=inputs.shape[0]*NEGSAMPLES, replacement=True)].reshape(inputs.shape[0], NEGSAMPLES)
            optimizer.zero_grad()
            loss = model.forward(inputs, targets, negatives)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress.set_postfix(loss=f"{loss.item():3.3f}")
        losses.append(total_loss)
        progress.set_postfix(loss=f"{total_loss:3.3f}")
    nearest_embedding_search('C0948008')
    
print(losses)


Epoch 1: 1520it [06:25,  3.94it/s, loss=8109.819]                                                                      


1.000	Ischemic stroke
0.405	lung; absence, acquired (any part)
0.396	IRRIGATION KIT W/PISTON SYRINGE K#68800
0.395	Calcification of pleura
0.391	Malignant neoplasm of anorectum
0.387	Difficulty verbalizing
0.386	Healing pressure ulcer of right heel NOS
0.380	Longitudinal absence of radius AND ulna
0.373	injury of multiple spinal nerve roots
0.372	functional limitations in self-care


Epoch 2: 1520it [06:25,  3.94it/s, loss=7390.028]                                                                      


1.000	Ischemic stroke
0.407	Difficulty verbalizing
0.404	IRRIGATION KIT W/PISTON SYRINGE K#68800
0.404	lung; absence, acquired (any part)
0.382	Ensure Plus Therapeutic Nutrition
0.380	Calcification of pleura
0.380	coronary angiography: coronary artery bypass graft origin left internal mammary artery insertion site visualized proximal left anterior descending
0.380	Utilize standardized abbreviations, acronyms, and symbols
0.375	Nemaline Myopathy 2
0.368	Patchy erythematous esophageal mucosa


Epoch 3: 1520it [06:13,  4.06it/s, loss=6016.769]                                                                      


...

1.000	Ischemic stroke
0.928	Acute Cerebrovascular Accidents
0.914	Acute ischemic stroke subtype
0.886	Right sided cerebral hemisphere cerebrovascular accident
0.886	CANCER WORKUP
0.884	neurological weakness of the right or left side
0.884	WEAKNESS RESIDUAL
0.883	CVA DISTRIBUTION MCA
0.881	Transient Ischemic Attack
0.877	Evaluation


Epoch 49: 1520it [10:23,  2.44it/s, loss=2456.504]                                                                     


1.000	Ischemic stroke
0.929	Acute Cerebrovascular Accidents
0.920	Acute ischemic stroke subtype
0.894	Right sided cerebral hemisphere cerebrovascular accident
0.890	CVA DISTRIBUTION MCA
0.889	WEAKNESS RESIDUAL
0.885	Transient Ischemic Attack
0.883	Cerebellar stroke
0.881	CANCER WORKUP
0.879	neurological weakness of the right or left side


Epoch 50: 1520it [10:55,  2.32it/s, loss=2475.188]                                                                     


1.000	Ischemic stroke
0.931	Acute Cerebrovascular Accidents
0.916	Acute ischemic stroke subtype
0.896	Right sided cerebral hemisphere cerebrovascular accident
0.888	WEAKNESS RESIDUAL
0.887	Right hemiparesis
0.887	Cerebellar stroke
0.887	Transient Ischemic Attack
0.884	Thalamic infarction
0.881	neurological weakness of the right or left side
[50915870.98339844, 36796070.26855469, 31732864.60107422, 28845594.072265625, 26832928.893554688, 25347657.813476562, 24173422.64794922, 23201841.671875, 22371291.514648438, 21653365.00048828, 21013578.751953125, 20448018.418945312, 19910806.518554688, 19436181.10986328, 18981112.2734375, 18577459.768798828, 18193587.456054688, 17817602.975341797, 17505028.23779297, 17173479.548583984, 16884315.399169922, 16590695.16772461, 16326634.245361328, 16075272.467041016, 15836793.860839844, 15610719.074951172, 15379732.203613281, 15179499.887207031, 14978053.685546875, 14792911.081054688, 14611129.380126953, 14435224.587402344, 14270305.335449219, 14098318.

In [17]:
torch.save(model.state_dict(), 'cui_model_11_MAR_2022.pt')

In [18]:
for _, i in enumerate(losses):
    if losses[_-1] < i: #if loss is always monitonically decreasing, should print 0 only
        print(_)

0


In [19]:
with open("cui_vectors_11_MAR_2022.json",'w') as outfile:
    json.dump({key:model.in_embeddings.weight.data[value].tolist() for key,value in vocab.items()}, outfile)

In [15]:
# ToDo: Build document vectors from CUI vector components
docvectors = np.zeros((len(cuidata), EMBEDDING_DIM))
for i,doc in tqdm(enumerate(cuidata), total=len(cuidata)):
    for cui in cuidata[doc]['cuis']:
        docvectors[i] += model.in_embeddings.weight.data[vocab[cui]].numpy()


100%|███████████████████████████████████| 29865/29865 [01:16<00:00, 391.28it/s]


In [21]:
# ToDo: Build document vectors from CUI vector components
docvectors = np.zeros((len(cuidata), EMBEDDING_DIM))
for i,doc in tqdm(enumerate(cuidata), total=len(cuidata)):
    tmpvecs = np.zeros(EMBEDDING_DIM)
    for cui in cuidata[doc]['cuis']:
        tmpvecs += model.in_embeddings.weight.data[vocab[cui]].numpy()
    docvectors[i] = tmpvecs / len(cuidata[doc]['cuis'])

100%|███████████████████████████████████| 29865/29865 [01:07<00:00, 439.69it/s]


In [22]:
with open('average_document_vectors_28_FEB_2022.json','w') as outfile:
    json.dump({x:list(y) for x,y in zip(cuidata.keys(), docvectors)}, outfile)

In [27]:
'''
To load the saved pytorch model:

model = CUIEmbeddingModel(len(vocab), EMBEDDING_DIM) #vocab and embedding dim must match what the model was trained on
model.load_state_dict(torch.load(cui_model_15_NOV_2021.pt))
model.eval() #set to evaluation mode
''';

In [32]:
nearest_embedding_search('C0742946')

1.000	CVA ETIOLOGY HEMORRHAGIC ISCHEMIC
0.847	Acute Cerebrovascular Accidents
0.836	Ischemic stroke
0.829	Left hemiparesis
0.829	Structure of middle cerebral artery
0.822	Thalamic infarction
0.821	Transient Ischemic Attack
0.820	Evaluation
0.815	Acute ischemic stroke subtype
0.815	transient ischemic attack without residual deficits


In [33]:
import xgboost